In [21]:
import requests
from bs4 import BeautifulSoup
import os
import csv

In [147]:
def extract_cars(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    clase_sup = soup.find_all("lo", {"class":'ui-search-layout ui-search-layout--grid'})
    divs = soup.find_all("div", {"class":'andes-card poly-card poly-card--grid-card andes-card--flat andes-card--padding-0 andes-card--animated'})
    # print(divs)
    carros = []
    for item in divs:
        dict_list = {}
        clase_alta = BeautifulSoup(str(item),'html.parser')
        nombre_carro = clase_alta.find('h2', class_='poly-box poly-component__title').text
        precio_carro = clase_alta.find('span', class_='andes-money-amount__fraction').text
        imagenes = clase_alta.find_all('img', class_="poly-component__picture")
        for img in imagenes:
            url_imagen = img.get('src')
            if not url_imagen.endswith('.webp'):
                url_imagen = img.get('data-src')
            else:
                url_imagen = img.get('src')
        dict_list['Nombre'] = nombre_carro
        dict_list['Precio'] = precio_carro
        dict_list['Imagen_URL'] = url_imagen
        carros.append(dict_list)
    return carros

In [166]:
url = 'https://carros.tucarro.com.co/_Desde_289_PublishedToday_YES_NoIndex_True'
carros = extract_cars(url)

In [167]:
carros
with open('carros.csv', mode='w', newline='') as file: 
    writer = csv.writer(file)
    writer.writerow(['Nombre_Imagen', 'Precio'])
    for item in carros:
        try:
            nombre_imagen = item['Nombre'].replace(' ', '_') + '.webp'
            ruta_imagen = os.path.join('imagenes_carros', nombre_imagen)
            response = requests.get(item['Imagen_URL'])
            response.raise_for_status()
            with open(ruta_imagen, 'wb') as f:
                f.write(response.content)
            writer.writerow([nombre_imagen, item['Precio']])
        except Exception as e:
            continue